In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#데이터 불러오기
df = pd.read_csv('./data/samsung.csv', encoding="cp949")

In [3]:
#데이터의 구성을 확인하기 
df.head()

,일자,시가,고가,저가,종가,거래량
0,2020-01-31,"57,800","58,400","56,400","56,400","19,749,457"
1,2020-01-30,"58,800","58,800","56,800","57,200","20,821,939"
2,2020-01-29,"59,100","59,700","58,800","59,100","16,446,102"
3,2020-01-28,"59,400","59,400","58,300","58,800","23,664,541"
4,2020-01-23,"61,800","61,800","60,700","60,800","14,916,555"


In [5]:
#데이터가 얼마나 있는지 확인하기
df.tail()

,일자,시가,고가,저가,종가,거래량
421,2018-05-11,"52,000","52,200","51,200","51,300","10,314,997"
422,2018-05-10,"51,700","51,700","50,600","51,600","13,905,263"
423,2018-05-09,"52,600","52,800","50,900","50,900","16,128,305"
424,2018-05-08,"52,600","53,200","51,900","52,600","23,104,720"
425,2018-05-04,"53,000","53,900","51,800","51,900","39,565,391"


In [6]:
#날짜를 기준으로 데이터 정렬해주기
df=df.sort_values(['일자'], ascending = [True])

In [7]:
#데이터 값들이 전부 문자열이므로 숫자들로 바꿔주기
for col in df.columns[1:]:  # Iterate over chosen columns
    df[col] = df[col].str.replace(",","")
    df[col] = pd.to_numeric(df[col])

In [8]:
#데이터 나누기
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size = 0.2, shuffle  = False) #train : 298행  / test : 128행

In [9]:
train, validation = train_test_split(train, test_size = 0.2, shuffle  = False)

In [10]:
#일자를 기준으로 순서 정렬
train = train.sort_values(['일자'], ascending = [True])
train.head()

,일자,시가,고가,저가,종가,거래량
425,2018-05-04,53000,53900,51800,51900,39565391
424,2018-05-08,52600,53200,51900,52600,23104720
423,2018-05-09,52600,52800,50900,50900,16128305
422,2018-05-10,51700,51700,50600,51600,13905263
421,2018-05-11,52000,52200,51200,51300,10314997


In [11]:
#'일자'열은 학습에 필요없으니 지워주기
del train['일자']

In [12]:
#'일자'가 사라졌는지 확인하기
train

,시가,고가,저가,종가,거래량
425,53000,53900,51800,51900,39565391
424,52600,53200,51900,52600,23104720
423,52600,52800,50900,50900,16128305
422,51700,51700,50600,51600,13905263
421,52000,52200,51200,51300,10314997
...,...,...,...,...,...
158,44800,45000,44550,44850,6664872
157,44800,45050,44300,44600,8607439
156,44200,44400,43400,43750,16906541
155,43750,44150,43300,44000,9322873


In [13]:
#train에서 학습용 y_train 분리
y_train = train['종가']

In [14]:
#train에서 학습용 y_train 분리한 다음 '종가'열 지워주기
del train['종가']

In [15]:
#train에서 종가열이 사라졌으니 x_train으로 이름 변경
x_train = train

In [16]:
#Validation 데이터도 마찬가지
y_val = validation['종가']

In [17]:
del validation['종가']

In [18]:
x_val = validation

In [19]:
del x_val['일자']

In [20]:
#일자를 기준으로 순서 정렬
test = test.sort_values(['일자'], ascending = [True])
test.head()

,일자,시가,고가,저가,종가,거래량
85,2019-09-25,49200,49350,48800,48900,9187141
84,2019-09-26,49000,49250,48900,49200,8494756
83,2019-09-27,48000,48700,48000,48400,8048041
82,2019-09-30,48050,49250,47900,49050,9497119
81,2019-10-01,48900,49100,48650,48850,6206035


In [21]:
y_test = test['종가']

In [22]:
del test['종가']

In [23]:
x_test = test

In [24]:
del x_test['일자']

In [25]:
#Keras 모듈 불러오기
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py

In [42]:
#모델 구성하기
model = Sequential()#---------------------모델 방식

model.add(Dense(10, input_shape = (4,))) #-Hidden Layer 1
model.add(Dense(15))#---------------------Hidden Layer 2
model.add(Dense(8)) #---------------------Hidden Layer 3
model.add(Dense(5)) #---------------------Hidden Layer 4
model.add(Dense(3)) #---------------------Hidden Layer 5
model.add(Dense(1)) #---------------------Output Layer

model.summary() #-------------------------레이어 형태&파라미터

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 10)                50        
_________________________________________________________________
dense_14 (Dense)             (None, 15)                165       
_________________________________________________________________
dense_15 (Dense)             (None, 8)                 128       
_________________________________________________________________
dense_16 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_17 (Dense)             (None, 3)                 18        
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 4         
Total params: 410
Trainable params: 410
Non-trainable params: 0
________________________________________________________

In [43]:
#학습하기
model.compile(loss = 'mse', optimizer='adam', metrics=['mse']) 
model.fit(x_train, y_train, epochs = 200, batch_size=3, validation_data=(x_val, y_val)) 

Train on 272 samples, validate on 68 samples
Epoch 1/200
272/272 [==============================] - 1s 4ms/step - loss: 771042883762.7352 - mse: 771042705408.0000 - val_loss: 3406492491.2941 - val_mse: 3406492160.0000
Epoch 2/200
272/272 [==============================] - 0s 723us/step - loss: 1377954421.2794 - mse: 1377954432.0000 - val_loss: 4362780397.1765 - val_mse: 4362780160.0000
Epoch 3/200
272/272 [==============================] - 0s 749us/step - loss: 2509172264.8676 - mse: 2509172736.0000 - val_loss: 580045123.0588 - val_mse: 580045056.0000
Epoch 4/200
272/272 [==============================] - 0s 784us/step - loss: 3562160493.9926 - mse: 3562160128.0000 - val_loss: 1666039240.4706 - val_mse: 1666039168.0000
Epoch 5/200
272/272 [==============================] - 0s 735us/step - loss: 3405079144.0515 - mse: 3405079808.0000 - val_loss: 437947302.1765 - val_mse: 437947264.0000
Epoch 6/200
272/272 [==============================] - 0s 740us/step - loss: 4035541042.0882 - mse: 40

Epoch 49/200
272/272 [==============================] - 0s 684us/step - loss: 983946545.4099 - mse: 983946560.0000 - val_loss: 24893827.2169 - val_mse: 24893828.0000
Epoch 50/200
272/272 [==============================] - 0s 706us/step - loss: 380525268.6507 - mse: 380525184.0000 - val_loss: 84080825.9412 - val_mse: 84080824.0000
Epoch 51/200
272/272 [==============================] - 0s 664us/step - loss: 2478839681.1075 - mse: 2478839552.0000 - val_loss: 520828397.1765 - val_mse: 520828384.0000
Epoch 52/200
272/272 [==============================] - 0s 691us/step - loss: 1008499322.2013 - mse: 1008499200.0000 - val_loss: 1205288504.0000 - val_mse: 1205288320.0000
Epoch 53/200
272/272 [==============================] - 0s 697us/step - loss: 186159713.7911 - mse: 186159728.0000 - val_loss: 59466228.0588 - val_mse: 59466228.0000
Epoch 54/200
272/272 [==============================] - 0s 696us/step - loss: 1082019393.5699 - mse: 1082019200.0000 - val_loss: 1063032250.8235 - val_mse: 1063

272/272 [==============================] - 0s 702us/step - loss: 198602431.7942 - mse: 198602464.0000 - val_loss: 14588221.0294 - val_mse: 14588221.0000
Epoch 99/200
272/272 [==============================] - 0s 668us/step - loss: 209794158.3197 - mse: 209794208.0000 - val_loss: 509998.5041 - val_mse: 509998.5312
Epoch 100/200
272/272 [==============================] - 0s 730us/step - loss: 11889219.1336 - mse: 11889219.0000 - val_loss: 2303698.6438 - val_mse: 2303698.5000
Epoch 101/200
272/272 [==============================] - 0s 784us/step - loss: 29257643.8362 - mse: 29257644.0000 - val_loss: 3434230.2509 - val_mse: 3434230.2500
Epoch 102/200
272/272 [==============================] - 0s 742us/step - loss: 131716863.4988 - mse: 131716864.0000 - val_loss: 1319030940.7059 - val_mse: 1319030912.0000
Epoch 103/200
272/272 [==============================] - 0s 737us/step - loss: 25309766909.6121 - mse: 25309759488.0000 - val_loss: 78304521.2941 - val_mse: 78304520.0000
Epoch 104/200
272

Epoch 148/200
272/272 [==============================] - 0s 742us/step - loss: 535163013.6438 - mse: 535162880.0000 - val_loss: 91060773.1176 - val_mse: 91060768.0000
Epoch 149/200
272/272 [==============================] - 0s 765us/step - loss: 164969393.8640 - mse: 164969344.0000 - val_loss: 13948963.5993 - val_mse: 13948964.0000
Epoch 150/200
272/272 [==============================] - 0s 860us/step - loss: 155813761.8406 - mse: 155813808.0000 - val_loss: 126475212.3529 - val_mse: 126475216.0000
Epoch 151/200
272/272 [==============================] - 0s 715us/step - loss: 134341142.0517 - mse: 134341168.0000 - val_loss: 114657721.1765 - val_mse: 114657728.0000
Epoch 152/200
272/272 [==============================] - 0s 683us/step - loss: 248680775.6915 - mse: 248680800.0000 - val_loss: 48873943.1765 - val_mse: 48873948.0000
Epoch 153/200
272/272 [==============================] - 0s 681us/step - loss: 211268761.3061 - mse: 211268736.0000 - val_loss: 17526196.3640 - val_mse: 17526196

Epoch 198/200
272/272 [==============================] - 0s 748us/step - loss: 261709526.9467 - mse: 261709520.0000 - val_loss: 118994108.0294 - val_mse: 118994104.0000
Epoch 199/200
272/272 [==============================] - 0s 742us/step - loss: 140374941.2663 - mse: 140374944.0000 - val_loss: 25147664.8015 - val_mse: 25147666.0000
Epoch 200/200
272/272 [==============================] - 0s 811us/step - loss: 124983092.7206 - mse: 124983104.0000 - val_loss: 187116744.7059 - val_mse: 187116752.0000


In [44]:
#평가하기
loss, mse = model.evaluate(x_test,y_test,batch_size=1)
print('mse:', mse)

86/86 [==============================] - 0s 769us/step
mse: 256176128.0


In [49]:
#예측하기
x_prd = np.array([57100, 58800, 56800, 23995260])
test = model.predict(x_prd, batch_size=4)
print(test)

ValueError: Error when checking input: expected dense_13_input to have shape (4,) but got array with shape (1,)